# 例子01： 利用theano包 演示两个标量相加
## $x=1$, $y=3$, $z=x+y=1+3=4$

In [51]:
# example 01
import numpy
import theano.tensor as T
from theano import function
x = T.scalar('x')
y = T.scalar('y')
z = x + y
g = function([x, y], z)
g(1, 3)

array(4.0)

# 例子02 利用theano包 演示两个矩阵相加
## $x=\left[ \begin{matrix}
   1 & 2  \\
   3 & 4  \\
\end{matrix} \right],y=\left[ \begin{matrix}
   8 & 6  \\
   7 & 15  \\
\end{matrix} \right]$
## 结果自然就是 $z=\left[ \begin{matrix}
   9 & 8  \\
   10 & 19  \\
\end{matrix} \right]$

In [52]:
#example 02 
import numpy
import theano.tensor as T
from theano import function
x = T.dmatrix('x')
y = T.dmatrix('y')
z = x + y
g = function([x, y], z)
print(x)
print(y)
print(g([[1, 2], [3, 4]], [[8, 6], [7, 15]]))

x
y
[[  9.   8.]
 [ 10.  19.]]


# 例子03 使用Theano的矢量功能
## 我们定义了一个函数 $out=a+a^{3}+2$
## 我们让 $a=[0,1,2,3,4]$
## 读者可以自行验证结果
## $out[0]=0+0^{3}+2=2$
## $out[1]=1+1^{3}+2=4$
## $out[2]=2+2^{3}+2=2+8+2=12$
## $out[3]=3+3^{3}+2=3+27+2=32$
## $out[4]=4+4^{3}+2=4+4*4*4+2=4+64+2=70$

In [53]:
#example 03
import theano
a = theano.tensor.vector() # declare variable
out = a + a ** 3+ 2               # build symbolic expression
f = theano.function([a], out)   # compile function
print(f([0, 1, 2, 3, 4]))

[  2.   4.  12.  32.  70.]


# 例子04 下面我们定义一个矩阵$X$，然后计算矩阵的logistic函数
## 其中 ${{s}_{ij}}=\frac{1}{1+\exp \left( -{{x}_{ij}} \right)}$
## $X=\left[ {{x}_{ij}} \right]=\left[ \begin{matrix}
   0 & 1  \\
   -1 & -2  \\
\end{matrix} \right]$
## 输出结果
## $S=\left[ {{s}_{ij}} \right]=\left[ \begin{matrix}
   0.5 & 0.73105858  \\
   0.26894142 & 0.11920292  \\
\end{matrix} \right]$


In [44]:
# example 04 
import theano
import theano.tensor as T
x = T.dmatrix('x')
s = 1 / (1 + T.exp(-x))
logistic = theano.function([x], s)
logistic([[0, 1], [-1, -2]])

array([[ 0.5       ,  0.73105858],
       [ 0.26894142,  0.11920292]])

# 例子05 更为真实的例子 Logistic回归 例子来自于：theano官方网站
## 目的是要说明：在windows下 theano 可正常运行

In [42]:
# example 05
import numpy
import theano
import theano.tensor as T
rng = numpy.random

N = 400                                   # training sample size
feats = 784                               # number of input variables

# generate a dataset: D = (input_values, target_class)
D = (rng.randn(N, feats), rng.randint(size=N, low=0, high=2))
training_steps = 10000

# Declare Theano symbolic variables
x = T.dmatrix("x")
y = T.dvector("y")

# initialize the weight vector w randomly
#
# this and the following bias variable b
# are shared so they keep their values
# between training iterations (updates)
w = theano.shared(rng.randn(feats), name="w")

# initialize the bias term
b = theano.shared(0., name="b")

print("Initial model:")
print(w.get_value())
print(b.get_value())

# Construct Theano expression graph
p_1 = 1 / (1 + T.exp(-T.dot(x, w) - b))   # Probability that target = 1
prediction = p_1 > 0.5                    # The prediction thresholded
xent = -y * T.log(p_1) - (1-y) * T.log(1-p_1) # Cross-entropy loss function
cost = xent.mean() + 0.01 * (w ** 2).sum()# The cost to minimize
gw, gb = T.grad(cost, [w, b])             # Compute the gradient of the cost
                                          # w.r.t weight vector w and
                                          # bias term b
                                          # (we shall return to this in a
                                          # following section of this tutorial)

# Compile
train = theano.function(
          inputs=[x,y],
          outputs=[prediction, xent],
          updates=((w, w - 0.1 * gw), (b, b - 0.1 * gb)))
predict = theano.function(inputs=[x], outputs=prediction)

# Train
for i in range(training_steps):
    pred, err = train(D[0], D[1])

print("Final model:")
print(w.get_value())
print(b.get_value())
print("target values for D:")
print(D[1])
print("prediction on D:")
print(predict(D[0]))

Initial model:
[ -1.01060375e+00   4.36942194e-01   8.12410464e-01  -7.82253776e-01
   5.90775808e-01  -2.42027321e+00  -9.38185953e-01  -8.95429483e-01
  -6.10764503e-02   2.70785010e-01  -1.01229267e+00   6.01576507e-01
   2.47837076e+00  -8.63624509e-01  -5.35342638e-01  -8.80988961e-03
  -1.95380865e+00  -7.41244265e-01   1.24016934e-01  -5.53351343e-01
  -9.23027734e-01  -1.32472997e+00   1.70418393e+00   8.42978300e-01
  -3.66504582e-01   6.37792191e-01  -4.22309622e-01   7.76007388e-01
  -3.53330159e-01   3.33498950e-01   1.94140529e-01  -9.07790349e-01
   6.42653463e-01  -5.68270195e-01   1.14473663e+00  -3.97465618e-01
  -4.19077761e-01  -6.15306269e-01   7.92555757e-01   8.25075325e-01
   1.31608566e+00  -1.08814576e+00   2.84455942e-01  -3.97332316e-01
   1.14338494e+00  -5.11114593e-02   2.98589837e-01   1.15760047e-02
  -6.95613548e-01  -1.23892031e-02   1.29404248e+00   3.96246604e-01
   7.99313080e-01  -1.30215715e-01   5.10683654e-01  -2.01038903e-01
   1.09353988e+00  